In [1]:
!nvidia-smi

Tue Jan  7 21:51:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.77.01              Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   32C    P8             15W /  240W |    3028MiB /   8192MiB |     19%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch, time, os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import BertTokenizer

from src.Preprocessing import preprocessing_dataframe
from src.DataLoader import DataLoaderBert
from src.Model import Bert_FineTuned
from src.Callback import EarlyStopping
from src.Training import model_train, model_eval

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_LABELS = 5
RANDOM_SEED = 23
VOCAB_PATH = "vocab_file.txt"

DATA_PATH = os.path.join("data", "datos.xlsx")
DATA_EXT_PATH = os.path.join("data", "reviews_ext.xlsx") 
DATA_SIN = os.path.join("data", "datos_sinonimos.xlsx")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_excel(DATA_PATH)
column_name_rev = df.columns.to_list()[len(df.columns.to_list())-1]
columns_to_keep = df.columns.to_list()[:2]

df_revisado = df[df[column_name_rev] == 'Revisado'][columns_to_keep]
df_revisado_eq = preprocessing_dataframe(df_revisado, False)
df_revisado.shape, df_revisado_eq.shape

((6052, 2), (2220, 2))

In [4]:
df_ext = pd.read_excel(DATA_SIN)[columns_to_keep]
df_ext_eq = preprocessing_dataframe(df_ext, False)

df_eq = pd.concat([df_revisado_eq, df_ext_eq], axis=0)

df_ext_eq.shape, df_eq.shape

((33015, 2), (35235, 2))

In [5]:
df_train , df_test = train_test_split(df_eq, test_size=0.3, random_state = RANDOM_SEED)
df_val , df_test = train_test_split(df_test, test_size=0.5, random_state = RANDOM_SEED)
len(df_train), len(df_val), len(df_test)

(24664, 5285, 5286)

In [6]:
name_model = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
MAX_LEN = 512
BATCH_SIZE = 4
DROPOUT = 0.1

model = Bert_FineTuned(name_model, True, NUM_LABELS, DROPOUT).to(device)
model.load_state_dict(torch.load(os.path.join('model', 'bestBert.bin'), weights_only=True))

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [7]:
def get_predict(review):
    tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
    encoding = tokenizer.encode_plus(review,
                                          add_special_tokens=True, #Añade los tokens especiales que necesita BERT, [CLS] + [SEP]
                                          padding='max_length',  #Añade el pad para que todas las secuencias tengan la misma longitud
                                          truncation=True, #Trunca las secuancias más largas de 512, el BERT solo permite 512 tokens
                                          max_length=512,
                                          return_token_type_ids=False,  #Devuelve si un token pertenece a la primera o a la segunda secuencia
                                          return_attention_mask=True,
                                          return_tensors='pt')

    inputs_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    output_model = model(inputs_ids, attention_mask)
    preds = torch.argmax(output_model, dim = 1).tolist()
    return preds[0]

In [8]:
review = 'tenía ganas de ir a comer , me habían hablado súper bien del restaurante . si que es verdad , que el precio del menú del día me parece bastante excesivo 16 , 90 € . es cierto que la calidad , es buena , pero las cantidades no son muy abundantes para el precio que tiene el menú . el menú consta de 4 primeros 4 segundos y 3 postres , y muchos de los platos se repiten asiduamente en la carta . el trato normal , amables . la verdad , que no sería un localización al que desee volver , si se tercia pues iré , pero no especialmente . muy mal trato por pedazo de la camarera . eramos 5 ( con dos niños ) y nos pusieron en una mesa de dos , porque decían que las otras estaban reservadas . cuando llega nueva gente , la ponen en las otras mesas sin tener reserva . como íbamos con dos niños no iban a gastar dos mesas en nosotros . así nos lo ha dicho la camarera ( nuestros hijos también comen ) . nos hemos levantado e ido .'
get_predict(review)

0